In [ ]:
import tensorflow as tf
import numpy as np

class GF():
    def __init__(self, f):
        self.len_of_symbol = len(f)-1
        self.q = 2**self.len_of_symbol
        self.table, self.tf_table = np.array(self.make_code_table(f))
        
    def make_code_table(self, f):
        m = len(f)-1
        N = 2**m
        table = ["0"*m]
        tf_table = []
        x =""
        for i in range(1,m+1):
            if f[i]==0:
                x+="0"
            else:
                x+="1"
        for i in range(0, m):
            c="0"*m
            c = c[0:i] + "1"+c[i+1:]
            c = c[::-1]
            table.append(c)
        for i in range(m+1, N):
            a = table[-1]
            a1 = a[1:]+"0"
            b = "0"*m
            if a[0]=="1":
                b = x
            c = self.add_string_code(a1, b)
            table.append(c)
            vec = [False if ci == 0 else True for ci in c ]
            tf_vec = tf.constant(vec, dtype = tf.bool) 
            tf_table.append(tf_vec)
        return table, tf_table

    def add_string_code(self, a, b):
        c =""
        for i in range(self.len_of_symbol):
            if a[i]!= b[i] :
                c+="1"
            else:
                c+="0"
        return c
    
    def tensor(self, a):
        if len(a.shape) ==1:
            return self.tf_table(a)
        

In [ ]:
gl4 = GF([1, 1, 0, 0, 1])

In [ ]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.python.eager import context
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import check_ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops

def _flatten_outer_dims(logits):
  """Flattens logits' outer dimensions and keep its last dimension."""
  rank = array_ops.rank(logits)
  last_dim_size = array_ops.slice(
      array_ops.shape(logits), [math_ops.subtract(rank, 1)], [1])
  output = array_ops.reshape(logits, array_ops.concat([[-1], last_dim_size], 0))

  # Set output shape if known.
  if not context.executing_eagerly():
    shape = logits.get_shape()
    if shape is not None and shape.dims is not None:
      shape = shape.as_list()
      product = 1
      product_valid = True
      for d in shape[:-1]:
        if d is None:
          product_valid = False
          break
        else:
          product *= d
      if product_valid:
        output_shape = [product, shape[-1]]
        output.set_shape(output_shape)

  return output

def _swap_axis(logits, dim_index, last_index, name=None):
        """Swaps logits's dim_index and last_index."""
        return array_ops.transpose(
            logits,
            array_ops.concat([
                math_ops.range(dim_index), [last_index],
                math_ops.range(dim_index + 1, last_index), [dim_index]
            ], 0),
            name=name)
    

def reshape(logits, compute_op, dim=-1, name=None):
    logits = ops.convert_to_tensor(logits)
    # We need its original shape for shape inference.
    shape = logits.get_shape()
    is_last_dim = (dim is -1) or (dim == shape.ndims - 1)

    if shape.ndims is 2 and is_last_dim:
        return compute_op(logits, name=name)

    # If dim is the last dimension, simply reshape the logits to a matrix and
    # apply the internal softmax.
    if is_last_dim:
        input_shape = array_ops.shape(logits)
        logits = _flatten_outer_dims(logits)
        output = compute_op(logits)
        output = array_ops.reshape(output, input_shape, name=name)
        return output

    # If dim is not the last dimension, we have to do a reshape and transpose so
    # that we can still perform softmax on its last dimension.

    # Swap logits' dimension of dim and its last dimension.
    input_rank = array_ops.rank(logits)
    dim_axis = dim % shape.ndims
    logits = _swap_axis(logits, dim_axis, math_ops.subtract(input_rank, 1))
    shape_after_swap = array_ops.shape(logits)

    # Reshape logits into a matrix.
    logits = _flatten_outer_dims(logits)
    return logits,  shape_after_swap, dim_axis, compute_op, input_rank, shape

def unreshape(logits, shape_after_swap, dim_axis, compute_op, input_rank, shape):
    # Do the actual softmax on its last dimension.
    output = compute_op(logits)

    # Transform back the output tensor.
    output = array_ops.reshape(output, shape_after_swap)
    output = _swap_axis(
      output, dim_axis, math_ops.subtract(input_rank, 1))

    # Make shape inference work since reshape and transpose may erase its static
    # shape.
    output.set_shape(shape)

    return output

config=tf.ConfigProto(log_device_placement=True)
## GLOBAL CONFIG
TENSOR_DTYPE = tf.float16
NUMPY_DTYPE = np.float16
MaxIter = 15
batch_size = 3
A = 2
N = 6
K= 3
M = N - K
f = np.random.randint(1, 10, ( batch_size, A, N))
print(f)
print("#"*30)
## GRAPH
s1 = time.time()
P = tf.placeholder(TENSOR_DTYPE, [None, A, N], name= "Input")
L,  shape_after_swap, dim_axis, compute_op, input_rank, shape = reshape(P, gen_nn_ops.softmax, 1)
L = tf.transpose(L)
L = L/tf.reduce_sum(L, axis= 0)
L = tf.transpose(L)
L = unreshape(L, shape_after_swap, dim_axis, compute_op, input_rank, shape)
Q = tf.expand_dims(L, 1)   
Q = tf.tile(Q, [  1, M,  1, 1])
Q = tf.reshape(Q, [batch_size,   M, A,  N])
Z = tf.arg_min(Q, 2)
model = tf.global_variables_initializer()
sess =  tf.Session(config=config)
sess.run(model)

_L = sess.run(Q, feed_dict={P:f})
print(_L.shape)
print(_L)
s2 = time.time()
print(s2- s1)

In [62]:
import tensorflow as tf
import numpy as np
import time
config=tf.ConfigProto(log_device_placement=True)
A = 2
N = 6
K= 3
M = N-K
batch_size = 200
rand = np.random.randint(1, 10, ( batch_size, A, N))
#print(rand)
_GF2 = np.array([
    [1, 0],
    [0, 1]
])

In [63]:
tf.reset_default_graph()
GF2 = tf.constant(_GF2)
f = tf.placeholder(tf.float16, [None, A, N])
L =  tf.expand_dims(f, 1)   
Q =  tf.tile(L, [ 1, M, 1, 1]) 
Z = tf.arg_min(Q, dimension=2)
## Step 1
transposedQ = tf.transpose(Q, perm=[2, 0, 1, 3])
reshapedQ = tf.transpose(tf.reshape(transposedQ, [A, batch_size*M*N]))
Z2= tf.argmax(reshapedQ, axis= 1)
delQ_index_colum = tf.gather(GF2, Z2)
delQ_index_colum = tf.reshape(delQ_index_colum, [1, batch_size*A*M*N])
delQ_index_row = tf.transpose(tf.expand_dims(tf.range(batch_size*N*M), 0))
delQ_index_row = tf.tile(delQ_index_row, [1, A])
delQ_index_row = tf.reshape(delQ_index_row, [1, batch_size*A*N*M])
delQ_index =  tf.concat([delQ_index_row, delQ_index_colum], axis=0)
delQ_index = tf.transpose(delQ_index)
delQ_index = tf.reshape(delQ_index, [batch_size*M*N,  A, 2])
deltaQ = tf.gather_nd(reshapedQ, delQ_index)
deltaQ = tf.transpose(deltaQ)
deltaQ =tf.reshape(deltaQ, [A, batch_size, M, N])
deltaQ = tf.transpose(deltaQ, perm=[1, 2, 0, 3])

## Step 2
Beta = tf.reduce_sum(Z, axis= 2)

## Step 3







sess =  tf.Session(config=config)
s1 = time.time()
X = sess.run([Q, Z, Beta], feed_dict={f:rand})
s2 = time.time()
print (s2- s1)
for i in range(len(X)):
    print(X[i].shape)
    print(X[i])


0.2388594150543213
(200, 3, 2, 6)
[[[[7. 3. 4. 2. 3. 4.]
   [2. 9. 2. 6. 6. 8.]]

  [[7. 3. 4. 2. 3. 4.]
   [2. 9. 2. 6. 6. 8.]]

  [[7. 3. 4. 2. 3. 4.]
   [2. 9. 2. 6. 6. 8.]]]


 [[[4. 2. 5. 2. 5. 6.]
   [3. 3. 9. 8. 7. 2.]]

  [[4. 2. 5. 2. 5. 6.]
   [3. 3. 9. 8. 7. 2.]]

  [[4. 2. 5. 2. 5. 6.]
   [3. 3. 9. 8. 7. 2.]]]


 [[[3. 1. 5. 9. 9. 5.]
   [7. 7. 7. 5. 6. 4.]]

  [[3. 1. 5. 9. 9. 5.]
   [7. 7. 7. 5. 6. 4.]]

  [[3. 1. 5. 9. 9. 5.]
   [7. 7. 7. 5. 6. 4.]]]


 ...


 [[[4. 4. 8. 6. 6. 6.]
   [6. 4. 1. 4. 2. 9.]]

  [[4. 4. 8. 6. 6. 6.]
   [6. 4. 1. 4. 2. 9.]]

  [[4. 4. 8. 6. 6. 6.]
   [6. 4. 1. 4. 2. 9.]]]


 [[[1. 5. 2. 1. 5. 6.]
   [1. 6. 5. 1. 6. 4.]]

  [[1. 5. 2. 1. 5. 6.]
   [1. 6. 5. 1. 6. 4.]]

  [[1. 5. 2. 1. 5. 6.]
   [1. 6. 5. 1. 6. 4.]]]


 [[[3. 6. 6. 7. 3. 3.]
   [8. 9. 9. 4. 6. 5.]]

  [[3. 6. 6. 7. 3. 3.]
   [8. 9. 9. 4. 6. 5.]]

  [[3. 6. 6. 7. 3. 3.]
   [8. 9. 9. 4. 6. 5.]]]]
(200, 3, 6)
[[[1 0 1 0 0 0]
  [1 0 1 0 0 0]
  [1 0 1 0 0 0]]

 [[1 0 0 0 0 1]
  [1 0

In [ ]:
grap2 = grap.copy()
N = 120
for i in range(N):
    grap2[i] = (i+1)*grap[2]/grap[i]
import matplotlib.pyplot as plt
plt.plot(grap2[0:N])
plt.show()

In [ ]:
import tensorflow as tf
import time
import numpy as np
tf.reset_default_graph()
a = tf.constant(np.random.randint(-10, 10, (1000, 100)), tf.float32)
result = tf.reduce_max(a, axis=0)
sess =  tf.Session()
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
s1 = time.time()
_L = sess.run(result)
s2 = time.time()
print(_L.shape)
print(_L)
print(s2- s1)

In [ ]:
x = np.random.randint(0,3, (2))
print(x)
index = [1, 0]
print(x[index])